In [20]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt


In [21]:
df = yf.download("TCS.NS", start="2023-01-01", end="2023-12-31")
df.head()


/tmp/ipython-input-3831004128.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("TCS.NS", start="2023-01-01", end="2023-12-31")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,TCS.NS,TCS.NS,TCS.NS,TCS.NS,TCS.NS
Date,,,,,
2023-01-02,2986.108887,2991.053137,2962.074988,2985.696922,709547
2023-01-03,3031.795898,3039.715550,2971.321995,2977.456316,1245178
2023-01-04,3034.817627,3046.445635,3008.769502,3027.538834,1231668
2023-01-05,3031.567383,3055.555458,3004.878177,3049.054771,1826057
2023-01-06,2940.421631,3022.823568,2929.846671,3021.404379,2488376


In [22]:
def gap_up_down(df):
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)

    df['Gap_Up'] = df['Open'] > df['Prev_High']
    df['Gap_Down'] = df['Open'] < df['Prev_Low']
    return df


In [29]:
df = gap_up_down(df)
df[['Open','Prev_High','Prev_Low','Gap_Up','Gap_Down']].head()


ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(df.index, df['Close'], label='Close')

plt.scatter(df[df['Gap_Up']].index,
            df[df['Gap_Up']]['Close'],
            color='green', marker='^', label='Gap Up')

plt.scatter(df[df['Gap_Down']].index,
            df[df['Gap_Down']]['Close'],
            color='red', marker='v', label='Gap Down')

plt.legend()
plt.title("Gap Up and Gap Down Days – TCS (2023)")
plt.show()


In [ ]:
intraday = yf.download(
    "TCS.NS",
    interval="5m",
    period="5d",
    progress=False
)

intraday.columns = intraday.columns.get_level_values(0)
intraday.head()


In [ ]:
def calculate_vwap(df):
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['VWAP'] = (tp * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df


In [ ]:
intraday = calculate_vwap(intraday)


In [ ]:
plt.figure(figsize=(14,6))
plt.plot(intraday.index, intraday['Close'], label='Price')
plt.plot(intraday.index, intraday['VWAP'], label='VWAP', linewidth=2)

plt.title("Intraday VWAP – TCS")
plt.legend()
plt.show()


problem 1

RSI

In [ ]:
def rsi(df, period=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


BOLLINGER BANDS

In [ ]:
def bollinger_bands(df, window=20):
    df['BB_Middle'] = df['Close'].rolling(window).mean()
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['Close'].rolling(window).std()
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['Close'].rolling(window).std()
    return df


Macd

In [ ]:
def macd(df):
    ema12 = df['Close'].ewm(span=12).mean()
    ema26 = df['Close'].ewm(span=26).mean()

    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    return df


ADX


In [ ]:
def adx(df, period=14):
    high, low, close = df['High'], df['Low'], df['Close']

    tr = pd.concat([
        high - low,
        (high - close.shift()).abs(),
        (low - close.shift()).abs()
    ], axis=1).max(axis=1)

    df['ATR_ADX'] = tr.rolling(period).mean()
    df['ADX'] = 100 * (df['ATR_ADX'] / df['ATR_ADX'].rolling(period).mean())
    return df


Stochastic Oscillator

In [ ]:
def stochastic(df, period=14):
    low_min = df['Low'].rolling(period).min()
    high_max = df['High'].rolling(period).max()

    df['%K'] = 100 * (df['Close'] - low_min) / (high_max - low_min)
    df['%D'] = df['%K'].rolling(3).mean()
    return df


ATR

In [ ]:
def atr(df, period=14):
    tr = pd.concat([
        df['High'] - df['Low'],
        (df['High'] - df['Close'].shift()).abs(),
        (df['Low'] - df['Close'].shift()).abs()
    ], axis=1).max(axis=1)

    df['ATR'] = tr.rolling(period).mean()
    return df


problem 2

RSI Signal

In [24]:
def rsi_signal(df):
    df['RSI_Signal'] = np.where(df['RSI'] < 30, 'BUY',
                         np.where(df['RSI'] > 70, 'SELL', 'HOLD'))
    return df


Macd signal

In [25]:
def macd_signal(df):
    df['MACD_Trade'] = np.where(df['MACD'] > df['MACD_Signal'], 'BUY', 'SELL')
    return df


Bollinger Band Signals

In [26]:
def bb_signal(df):
    df['BB_Trade'] = np.where(df['Close'] < df['BB_Lower'], 'BUY',
                        np.where(df['Close'] > df['BB_Upper'], 'SELL', 'HOLD'))
    return df


Apply Signals

In [27]:
df = rsi_signal(df)
df = macd_signal(df)
df = bb_signal(df)


KeyError: 'RSI'

Signal Counts (Required Output)

In [ ]:
print("RSI Signals:\n", df['RSI_Signal'].value_counts())
print("\nMACD Signals:\n", df['MACD_Trade'].value_counts())
print("\nBollinger Band Signals:\n", df['BB_Trade'].value_counts())
